# Regression Exercise 

California Housing Data

This data set contains information about all the block groups in California from the 1990 Census. In this sample a block group on average includes 1425.5 individuals living in a geographically compact area. 

The task is to aproximate the median house value of each block from the values of the rest of the variables. 

 It has been obtained from the LIACC repository. The original page where the data set can be found is: http://www.liaad.up.pt/~ltorgo/Regression/DataSets.html.
 

The Features:
 
* housingMedianAge: continuous. 
* totalRooms: continuous. 
* totalBedrooms: continuous. 
* population: continuous. 
* households: continuous. 
* medianIncome: continuous. 
* medianHouseValue: continuous. 

## The Data

** Import the cal_housing_clean.csv file with pandas. Separate it into a training (70%) and testing set(30%).**

In [3]:
import pandas as pd
houses = pd.read_csv('cal_housing_clean.csv')

In [6]:
houses.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [4]:
from sklearn.model_selection import train_test_split

In [8]:
#Separating features and labels
x_houses = houses.drop('medianHouseValue', axis=1)

In [9]:
labels = houses['medianHouseValue']

In [22]:
X_train, X_test, y_train, y_test = train_test_split(x_houses, labels, test_size=0.3, random_state=101)

,count,mean,std,min,25%,50%,75%,max
housingMedianAge,20640.0,28.639486,12.585558,1.0000,18.0000,29.0000,37.00000,52.0000
totalRooms,20640.0,2635.763081,2181.615252,2.0000,1447.7500,2127.0000,3148.00000,39320.0000
totalBedrooms,20640.0,537.898014,421.247906,1.0000,295.0000,435.0000,647.00000,6445.0000
population,20640.0,1425.476744,1132.462122,3.0000,787.0000,1166.0000,1725.00000,35682.0000
households,20640.0,499.539680,382.329753,1.0000,280.0000,409.0000,605.00000,6082.0000
medianIncome,20640.0,3.870671,1.899822,0.4999,2.5634,3.5348,4.74325,15.0001
medianHouseValue,20640.0,206855.816909,115395.615874,14999.0000,119600.0000,179700.0000,264725.00000,500001.0000


### Scale the Feature Data

** Use sklearn preprocessing to create a MinMaxScaler for the feature data. Fit this scaler only to the training data. Then use it to transform X_test and X_train. Then use the scaled X_test and X_train along with pd.Dataframe to re-create two dataframes of scaled data.**

In [11]:
from sklearn.preprocessing import MinMaxScaler

In [23]:
scaler_model = MinMaxScaler()

In [24]:
# only on training we do not use test as that would assume that we had this prior knowledge
scaler_model.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [25]:
xtestrans = pd.DataFrame(scaler_model.transform(X_test)).head()

In [27]:
xtestrans.columns = x_houses.columns
xtestrans

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome
0,0.686275,0.046264,0.045158,0.025873,0.048841,0.353133
1,0.705882,0.027417,0.020795,0.012709,0.023187,0.770182
2,0.901961,0.032326,0.040813,0.041662,0.042592,0.133626
3,0.313725,0.043212,0.046089,0.032840,0.048018,0.263576
4,0.411765,0.088433,0.069367,0.043728,0.072192,0.660046


In [29]:
xtraintrans = pd.DataFrame(scaler_model.transform(X_train))
xtraintrans.columns = x_houses.columns
xtraintrans.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome
0,0.352941,0.069688,0.117163,0.048769,0.115442,0.142508
1,0.607843,0.011242,0.015673,0.008367,0.014142,0.045027
2,0.666667,0.025230,0.031347,0.020971,0.030258,0.212866
3,0.666667,0.032530,0.033830,0.024752,0.030094,0.298651
4,0.294118,0.031919,0.035692,0.019466,0.034863,0.272631


### Create Feature Columns

** Create the necessary tf.feature_column objects for the estimator. They should all be trated as continuous numeric_columns. **

In [30]:
import tensorflow as tf

In [46]:
# as they are all the same, they are all created using a for loop for brevity
feat_cols = []
for cols in x_houses.columns:
    feat_cols.append(tf.feature_column.numeric_column(cols))

In [47]:
feat_cols

[_NumericColumn(key='housingMedianAge', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='totalRooms', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='totalBedrooms', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='population', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='households', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='medianIncome', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

** Create the input function for the estimator object. (play around with batch_size and num_epochs)**

In [37]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=10,num_epochs=1000,shuffle=True)
print(input_func)

<function pandas_input_fn.<locals>.input_fn at 0x000001F7E49FB2F0>


** Create the estimator model. Use a DNNRegressor. Play around with the hidden units! **

In [48]:
dnn_model = tf.estimator.DNNRegressor(hidden_units=[10,10,10],feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_tf_random_seed': 1, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_save_summary_steps': 100, '_keep_checkpoint_max': 5, '_model_dir': 'C:\\Users\\FRKSTE~1\\AppData\\Local\\Temp\\tmp3jpr9bc7', '_save_checkpoints_secs': 600, '_log_step_count_steps': 100}


##### ** Train the model for ~1,000 steps. (Later come back to this and train it for more and check for improvement) **

In [49]:
dnn_model.train(input_fn=input_func, steps=1000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\FRKSTE~1\AppData\Local\Temp\tmp3jpr9bc7\model.ckpt.
INFO:tensorflow:step = 1, loss = 3.89226e+11
INFO:tensorflow:global_step/sec: 412.124
INFO:tensorflow:step = 101, loss = 1.35254e+11 (0.248 sec)
INFO:tensorflow:global_step/sec: 459.602
INFO:tensorflow:step = 201, loss = 2.77616e+11 (0.223 sec)
INFO:tensorflow:global_step/sec: 443.263
INFO:tensorflow:step = 301, loss = 6.07307e+11 (0.222 sec)
INFO:tensorflow:global_step/sec: 474.923
INFO:tensorflow:step = 401, loss = 2.95307e+11 (0.211 sec)
INFO:tensorflow:global_step/sec: 424.398
INFO:tensorflow:step = 501, loss = 3.25019e+11 (0.233 sec)
INFO:tensorflow:global_step/sec: 447.241
INFO:tensorflow:step = 601, loss = 1.68124e+11 (0.224 sec)
INFO:tensorflow:global_step/sec: 457.494
INFO:tensorflow:step = 701, loss = 2.49281e+11 (0.220 sec)
INFO:tensorflow:global_step/sec: 435.518
INFO:tensorflow:step = 801, loss = 6.19556e+10 (0.231 sec)
INFO

** Create a prediction input function and then use the .predict method off your estimator model to create a list or predictions on your test data. **

In [50]:
pred_input_func = tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=10,num_epochs=1,shuffle=False)

In [65]:
predictions = dnn_model.predict(pred_input_func)

In [66]:
preds = list(predictions)

INFO:tensorflow:Restoring parameters from C:\Users\FRKSTE~1\AppData\Local\Temp\tmp3jpr9bc7\model.ckpt-1000


In [69]:
#preds

** Calculate the RMSE. You should be able to get around 100,000 RMSE (remember that this is in the same units as the label.) Do this manually or use [sklearn.metrics](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) **

In [73]:
final_preds = []

for pred in preds:
    final_preds.append(pred['predictions'])

In [74]:
from sklearn.metrics import mean_squared_error

In [75]:
mean_squared_error(y_test, final_preds)**0.5

162770.50664831657

The result above means that on average, we are 162.000 from the actual value of the house -- which I would say is relatively bad.

What you could do to help this in the right direction is:
* Try  linear regressor
* Train for more steps
*    etc.

97921.93181985477

# Great Job!